In [5]:
import csv
from os import listdir
from os.path import isfile, join
onlyfiles = [f for f in listdir('./data') if isfile(join('./data', f))]

In [6]:
import re
import json
from collections import defaultdict

In [7]:
selected_files =  ['realDonaldTrump', 'SecretaryCarson','MartinOMalley', 'JebBush',
             'BarackObama', 'HillaryClinton','BernieSanders']

In [51]:
text = list()
labels = list()
data_source = defaultdict(int)
total_count = 12924
not_count = 0
for file in onlyfiles:        
    with open('data/' + file, newline='', encoding="utf8") as csv_file:
        if file[:-11] not in selected_files and 'trump' not in file.lower():
            continue                
        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        if 'trump' not in file.lower():
            label = 'NoTrump'
            if not_count == total_count:
                continue
        else:
            label = 'Trump'        
        text_ind = 1
        if "MORE_TRUMP.csv" in file:
            text_ind = 0
            source = 'realDonaldTrump'
        else:
            source = file[:-11]        
        for row in csv_reader:            
            if line_count != 0:
                line = re.sub('[\s\n\t#@]+', ' ', row[text_ind]).strip()
                line = re.sub(r'http\S+', '', line).strip()            
                if len(line) == 0:
                    continue                    
                text.append(line)                
                labels.append(label)            
                data_source[source] += 1
                if label != 'Trump':
                    not_count += 1
                    if not_count == total_count:
                        break                
            line_count += 1
from sklearn.model_selection import train_test_split
train_data, test_data, train_labels, test_labels = \
train_test_split(text, labels, test_size=0.3, random_state=42)
with open("train.csv", "w", newline='', encoding='utf8') as csv_file:
    csv_writer = csv.writer(csv_file)
    for i in range(len(train_data)):
        csv_writer.writerow([train_data[i], train_labels[i]])
with open("dev.csv", "w", newline='', encoding='utf8') as csv_file:
    csv_writer = csv.writer(csv_file)
    for i in range(len(test_data)):
        csv_writer.writerow([test_data[i], test_labels[i]])
from collections import Counter    
print(Counter(labels))                

Counter({'Trump': 12924, 'NoTrump': 12924})


In [58]:
data_source

defaultdict(int,
            {'realDonaldTrump': 12924,
             'SecretaryCarson': 1242,
             'MartinOMalley': 2211,
             'JebBush': 2757,
             'BarackObama': 2880,
             'HillaryClinton': 1774,
             'BernieSanders': 2060})

## Word Difficulty clasification

In [83]:
import requests
import re
def get_word_difficulty(word):
    URL = "https://datayze.com/word-analyzer.php"      
    PARAMS = {'word':word}   
    # sending get request and saving the response as response object 
    r = requests.get(url = URL, params = PARAMS) 
    misspell_text = re.findall('<tr><td class="indented">Misspelled.*</tr>', str(r.content))
    misspell_value_text = re.findall('<span class="value">.* </span>', misspell_text[0])[0]   
    print(misspell_value_text)
    if 'Yes' in misspell_value_text:
        return None
    grade_text = re.findall('<td class="indented">Grade Level.*</td>', str(r.content))
    value_text = re.findall('<td><span class="value">.*</span></td></tr>', grade_text[0])[0]
    return value_text[24: value_text.index('</span>')]

In [53]:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')
from nltk.stem import PorterStemmer
ps = PorterStemmer() 

In [56]:
primary_list = set()
sat_list = set()
toefl_list = set()
gre_list = set()
with open('vocabulary/primary_vocab.txt', 'r') as f:    
    for line in f:
        if line is not None:
            words = tokenizer.tokenize(line)
            for i in range(len(words)):
                words[i] = ps.stem(words[i])
            primary_list.update(words)
with open('vocabulary/SAT_vocab.txt', 'r') as f:
    for line in f:
         if line is not None:
            words = tokenizer.tokenize(line)
            for i in range(len(words)):
                words[i] = ps.stem(words[i])
            sat_list.add(words[0])
with open('vocabulary/TOEFL_vocab.txt', 'r') as f:
    for line in f:
         if line is not None:
            words = tokenizer.tokenize(line)
            for i in range(len(words)):
                words[i] = ps.stem(words[i])
            toefl_list.update(words)
with open('vocabulary/GRE_vocab.txt', 'r') as f:
    for line in f:
         if line is not None:
            words = tokenizer.tokenize(line) 
            if len(words) == 0:
                continue
            for i in range(len(words)):
                words[i] = ps.stem(words[i])
            gre_list.add(words[0])

In [57]:
words_level = dict()
for file in onlyfiles:       
    with open('data/' + file, newline='', encoding="utf8") as csv_file:        
        if file[:-11] not in selected_files and 'trump' not in file.lower():
            continue     
        csv_reader = csv.reader(csv_file, delimiter=',')                
        text_ind = 1
        if "MORE_TRUMP.csv" in file:
            text_ind = 0
            source = 'realDonaldTrump'
        else:
            source = file[:-11]
        if source not in words_level:
            words_level[source] = defaultdict(int)
        line_count = 0        
        for row in csv_reader:
            if line_count != 0:
                line = re.sub('[\s\n\t#@]+', ' ', row[text_ind])
                line = re.sub(r'http\S+', '', line).strip()            
                if len(line) == 0:
                    continue
                words = tokenizer.tokenize(row[text_ind])
                for w in words:
                    w = ps.stem(w)                    
                    value = 'others'
                    if w in gre_list:
                        value = 'GRE' 
                    elif w in toefl_list:
                        value = 'TOEFL' 
                    elif w in primary_list:
                        value = 'primary'                                        
                    words_level[source][value] += 1
                    words_level[source]['total'] += 1
            line_count += 1
                    
        

In [59]:
words_level

{'realDonaldTrump': defaultdict(int,
             {'TOEFL': 124530,
              'total': 340958,
              'others': 54268,
              'primary': 145408,
              'GRE': 16752}),
 'SecretaryCarson': defaultdict(int,
             {'primary': 14905,
              'total': 37792,
              'TOEFL': 13765,
              'others': 7448,
              'GRE': 1674}),
 'MartinOMalley': defaultdict(int,
             {'primary': 19832,
              'total': 47548,
              'others': 11004,
              'TOEFL': 15060,
              'GRE': 1652}),
 'JebBush': defaultdict(int,
             {'primary': 23104,
              'total': 56243,
              'others': 13710,
              'TOEFL': 17147,
              'GRE': 2282}),
 'BarackObama': defaultdict(int,
             {'primary': 25305,
              'total': 58141,
              'TOEFL': 19206,
              'others': 11737,
              'GRE': 1893}),
 'HillaryClinton': defaultdict(int,
             {'primary': 20446

In [60]:
word_percent = dict()
for key, v_dict in words_level.items():    
    if key not in word_percent:
        word_percent[key] = defaultdict(float)
    for cate, count in v_dict.items():
        if cate == 'total':
            continue
        word_percent[key][cate] = count / words_level[key]['total']

In [61]:
word_percent

{'realDonaldTrump': defaultdict(float,
             {'TOEFL': 0.36523560086579576,
              'others': 0.1591632987054124,
              'primary': 0.42646894925474693,
              'GRE': 0.049132151174044895}),
 'SecretaryCarson': defaultdict(float,
             {'primary': 0.39439563928873833,
              'TOEFL': 0.3642305249788315,
              'others': 0.19707874682472482,
              'GRE': 0.04429508890770534}),
 'MartinOMalley': defaultdict(float,
             {'primary': 0.4170943047026163,
              'others': 0.2314292925044166,
              'TOEFL': 0.31673256498696056,
              'GRE': 0.03474383780600656}),
 'JebBush': defaultdict(float,
             {'primary': 0.410788898173995,
              'others': 0.2437636683676191,
              'TOEFL': 0.3048734953683125,
              'GRE': 0.04057393809007343}),
 'BarackObama': defaultdict(float,
             {'primary': 0.43523503207719166,
              'TOEFL': 0.33033487556113583,
              'other

In [63]:
import json
with open('statistic/words_level.json', 'w') as fp:
    json.dump(word_percent, fp)